Rodzaje ruchu:\
Swap - losujemy dwa miasta i zamieniamy je ze sobą.\
Insercja - losujemy miasto i punkt w którym je umieszczamy.\
Zamiana kolejności - losujemy dwa punkty i między nimi zamieniamy kolejność.\
\
Sąsiedztwo to dwa przypadki które różnią się między sobą jednym ruchem\
\
Klasyczny algorytm wspinaczki z multistartem: po każdym ruchu liczymy drogę i sprawdzamy czy jest mniejsza czy nie, wybieramy tylko lepsze opcje. Multistart polega na wielokrotnym odpaleniu algorytmu z losowaniem punktu startowego.\
\
Algorytm symulowanego wyżarzania: zmodyfikowany algorytm wspinaczki, jeśli znalezione rozwiązanie jest gorsze to nadal je rozważamy, na podstawie tego jak bardzo jest gorsze oraz od temperatury.\
Temperatura jest wysoka na początku działania algorytmu i spada razem z czasem wykonywania algorytmu, im niższa temperatura tym mniejsze prawdopodobieństwo wybrania gorszego rozwiązania, bo algorytm już trochę działa i zakładamy że jest w lepszym miejscu niż na początku.\
Pozwala on nam wyjść z niektórych ekstremów lokalnych dzięki możliwości przyjęcia gorszego rozwiązania.\
\
Tabu search:\
Sąsiedztwo wygląda tutaj inaczej: przy swapie losujemy jedną liczbę i zamieniamy wszystkie kombinacje z tą liczbą, ze wszystkich kombinacji wybieramy najlepszą.\
Następnie taka kombinacja trafia na listę tabu, czyli takiej samej zamiany nie można wykonać przez określoną kolejną liczbę ruchów.\
\
Funkcja aspiracji to funkcja, która pod pewnymi warunkami pozwala wykonać ruch z listy tabu ale nie trzeba tego implementować.\
\
Odpowiednia długość listy tabu pozwala wyjść z niektórych ekstremów lokalnych, jeśli będzie za krótka to możemy nie wyjść z ekstremum lokalnego, jeśli będzie za długa to możemy wyjść z ekstremum niepotrzebnie.


In [2]:
import numpy as np
import pandas as pd
import math
import random

In [3]:
#wspinaczka z multistartem
distance = np.array([[0,2,4,1,5],
                    [2,0,3,8,7],
                    [4,3,0,5,1],
                    [1,8,5,0,3],
                    [5,7,1,3,0]])

In [4]:
def Odleglosc(distance, trasa):
    suma = 0
    n = len(trasa)
    
    for i, current_city in enumerate(trasa):
        next_city = trasa[(i + 1) % n]  # Use modulo to handle the last element
        suma += distance[current_city][next_city]

    return suma

In [5]:
#swap
def Swap(trasa):
    trasa_temp = trasa.copy()
    #print(trasa)
    swap = np.random.choice(trasa_temp,2,replace=False)
    temp = trasa_temp[swap[0]]
    trasa_temp[swap[0]] = trasa_temp[swap[1]]
    trasa_temp[swap[1]] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

#insercja
def Insercja(trasa):
    trasa_temp = trasa.copy()
    insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[insercja[0]]
    trasa_temp = np.delete(trasa_temp,insercja[0])
    trasa_temp = np.insert(trasa_temp,insercja[1],temp)
    #print(trasa_temp)
    return trasa_temp

def Zamiana(trasa):
    trasa_temp = trasa.copy()
    zamiana = np.random.choice(trasa_temp,2,replace=False)
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

In [6]:
#Zamiana(trasa)

In [7]:
#trasa = np.arange(len(distance))
#np.random.shuffle(trasa)
#Insercja(trasa)

In [8]:
#temp = trasa[0] #temp = 4
#trasa = np.delete(trasa,0)
#trasa = np.insert(trasa,4,3)

In [9]:
#ruch polegający na swapie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_swap(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Swap(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl, trasa_new, odl_new

#ruch polegający na insercji, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_insercja(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Insercja(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl

#ruch polegający na zamianie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_zamiana(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Zamiana(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new  
    return trasa, odl

In [10]:
distance = pd.read_excel("Dane_TSP_48.xlsx")
distance = distance.to_numpy()
distance = np.delete(distance,0,1)
distance

FileNotFoundError: [Errno 2] No such file or directory: 'Dane_TSP_48.xlsx'

In [ ]:
def IHC(distance, ruch, n_iter_ruch, n_iter):  # ruch = {Swap, Insercja, Zamiana}
    min_iter = []
    for i in range(n_iter):
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                if odl_new < odl:
                    odl = odl_new
                    trasa = trasa_new
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = IHC(distance,Swap,1000,100)
wynik.sort(key=lambda x: x[2])
wynik[0:10]



NameError: name 'distance' is not defined

In [ ]:
def Geometric(T,alpha):
    return alpha*T

def Slow(T,alpha):
    return T/(1+alpha*T)


In [ ]:
def AS(distance, ruch, n_iter_ruch, n_iter, T_start, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = AS(distance,Swap,1000,100,100,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

[(90,
  array([43, 17, 30, 45, 19, 38, 47,  9, 25,  3, 34, 44, 23, 31,  4, 28,  1,
         41, 40, 33, 46, 10, 12, 20, 24, 13, 22,  7, 37,  6, 27, 26, 16, 42,
         29, 32, 14,  8,  0, 21, 15,  2, 39, 11,  5, 36, 18, 35]),
  15881),
 (10,
  array([32, 19, 42, 36,  5, 35, 45, 37, 39,  6, 30, 43, 27, 18, 16, 26, 29,
         17,  7,  2, 46, 20, 12, 47, 41,  9, 23, 31, 38, 24, 33,  8,  0, 21,
         15, 40, 28,  1, 25,  3, 34, 44,  4, 11, 22, 13, 10, 14]),
  16981),
 (13,
  array([44, 34,  9, 23, 41, 25,  1, 28, 21, 33, 40, 15,  7,  8,  0, 24,  4,
         47, 38, 31, 20, 46, 22, 10, 11, 32, 14, 39, 45, 27, 16, 26,  5, 17,
         43, 35, 36, 18,  6, 30, 37, 29, 42, 19,  2, 13, 12,  3]),
  16989),
 (1,
  array([15, 40,  1,  3, 25, 28,  4, 47, 38, 20, 42, 16, 18, 26,  6, 17, 43,
         45, 30, 37,  7,  0, 36, 35, 19,  8, 39, 29,  5, 27, 32, 11, 14, 46,
         12, 24, 10, 22, 13, 33, 31, 23, 44, 34,  9, 41,  2, 21]),
  17232),
 (88,
  array([35, 17, 26, 42, 16, 18,  6, 43, 36, 19

In [ ]:
# wersja AS gdzie warunkiem stopu jest temperatura a nie liczba iteracji 
# (chyba lepiej tak bo upewniamy sie ze temperatura jest niska a podajac liczbe iteracji mozemy skonczyc na wysokiej temperaturze)
# mozna tez dodac warunek stopu ze po np 100 iteracjach bez poprawy rozwiazania to koniec

def AS(distance, ruch, n_iter, T_start, T_stop, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        while(T > T_stop):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = AS(distance,Swap,100,100,0.01,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

[(84,
  array([11, 19,  4, 47,  1, 28,  3, 25, 41, 44, 34,  9, 23, 31, 38, 20, 24,
         15, 21,  0,  2, 40, 33, 13, 22, 12, 46, 10, 39,  8,  7, 37, 18, 36,
          5, 35, 30, 14, 17, 29, 42, 16, 26, 27,  6, 45, 43, 32]),
  15656),
 (72,
  array([41,  1, 25,  3, 44, 34, 23,  9, 31, 13, 24,  4, 47, 38, 19,  5, 29,
         27, 17, 35,  6, 18, 26, 42, 16, 36, 45, 21,  2, 10, 20, 46, 11, 32,
         14, 33, 40, 15,  0,  7, 43, 30, 37,  8, 39, 22, 12, 28]),
  16108),
 (76,
  array([24, 12, 20, 39, 37, 43, 17, 35, 26, 16, 42,  4, 28,  1,  3, 25, 31,
         23, 44, 34,  9, 41, 47, 38, 13, 33, 40,  2, 15,  0, 21, 22, 10, 45,
         30,  7,  8, 32, 19, 29, 18, 36,  5, 27,  6, 14, 11, 46]),
  16618),
 (14,
  array([21, 32,  5, 36, 26, 18, 16, 42, 43, 30, 45, 17,  6, 27, 35, 29, 37,
          8, 11, 38, 44, 34, 25,  3,  1, 47, 31, 23, 41,  9,  4, 28, 40, 33,
         24, 10, 14, 39, 22,  2, 13, 12, 20, 46, 19,  7,  0, 15]),
  16707),
 (27,
  array([43,  5, 18, 36, 29, 27, 35, 14, 11, 1

In [ ]:
# funkcje pomocnicze do sprawdzenia ile iteracji odpowiada parametrowi T_stop
def T_stopToIter(T_start, T_stop, cooling, alpha):
    T = T_start
    i = 0
    while(T > T_stop):
        T = cooling(T,alpha)
        i += 1
    return i
def IterToT_stop(T_start, n_iter, cooling, alpha):
    T = T_start
    for i in range(n_iter):
        T = cooling(T,alpha)
    return T
print(T_stopToIter(100,0.01,Slow,0.1))
IterToT_stop(100,100,Slow,0.1)

1000


0.09990009990009993

In [ ]:
#algorytm ze swapem
trasa = np.arange(len(distance))
np.random.shuffle(trasa)
licza_iteracji = 5

print("Start")
print(trasa)
print(Odleglosc(distance,trasa))
for i in range(licza_iteracji):
    print("\n{} iteracja".format(i+1))
    trasa = Ruch_swap(distance,trasa)
    print(trasa)
    print(Odleglosc(distance,trasa))


Start
[1 4 0 2 3]
29

1 iteracja
[1 0 4 2 3]
21

2 iteracja
[0 1 4 2 3]
16

3 iteracja
nowa trasa jest dluzsza lub rowna
[0 1 4 2 3]
16

4 iteracja
nowa trasa jest dluzsza lub rowna
[0 1 4 2 3]
16

5 iteracja
nowa trasa jest dluzsza lub rowna
[0 1 4 2 3]
16


In [ ]:
#algorytm z insercją
trasa = np.arange(len(distance))
np.random.shuffle(trasa)
licza_iteracji = 5

print("Start")
print(trasa)
print(Odleglosc(distance,trasa))
for i in range(licza_iteracji):
    print("\n{} iteracja".format(i+1))
    trasa = Ruch_insercja(distance,trasa)
    print(trasa)
    print(Odleglosc(distance,trasa))

Start
[2 1 4 0 3]
21

1 iteracja
[2 4 0 3 1]
18

2 iteracja
nowa trasa jest dluzsza lub rowna
[2 4 0 3 1]
18

3 iteracja
nowa trasa jest dluzsza lub rowna
[2 4 0 3 1]
18

4 iteracja
nowa trasa jest dluzsza lub rowna
[2 4 0 3 1]
18

5 iteracja
nowa trasa jest dluzsza lub rowna
[2 4 0 3 1]
18


In [ ]:
#algorytm z zamianą
trasa = np.arange(len(distance))
np.random.shuffle(trasa)
licza_iteracji = 1000

print("Start")
print(trasa)
print(Odleglosc(distance,trasa))
for i in range(licza_iteracji):
    print("\n{} iteracja".format(i+1))
    trasa = Ruch_zamiana(distance,trasa)
    print(trasa)
    print(Odleglosc(distance,trasa))

Start
[1 4 0 2 3]
29

1 iteracja
[1 4 2 0 3]
21

2 iteracja
nowa trasa jest dluzsza lub rowna
[1 4 2 0 3]
21

3 iteracja
nowa trasa jest dluzsza lub rowna
[1 4 2 0 3]
21

4 iteracja
[4 1 2 0 3]
18

5 iteracja
nowa trasa jest dluzsza lub rowna
[4 1 2 0 3]
18

6 iteracja
nowa trasa jest dluzsza lub rowna
[4 1 2 0 3]
18

7 iteracja
nowa trasa jest dluzsza lub rowna
[4 1 2 0 3]
18

8 iteracja
nowa trasa jest dluzsza lub rowna
[4 1 2 0 3]
18

9 iteracja
nowa trasa jest dluzsza lub rowna
[4 1 2 0 3]
18

10 iteracja
nowa trasa jest dluzsza lub rowna
[4 1 2 0 3]
18

11 iteracja
[4 2 1 0 3]
10

12 iteracja
nowa trasa jest dluzsza lub rowna
[4 2 1 0 3]
10

13 iteracja
nowa trasa jest dluzsza lub rowna
[4 2 1 0 3]
10

14 iteracja
nowa trasa jest dluzsza lub rowna
[4 2 1 0 3]
10

15 iteracja
nowa trasa jest dluzsza lub rowna
[4 2 1 0 3]
10

16 iteracja
nowa trasa jest dluzsza lub rowna
[4 2 1 0 3]
10

17 iteracja
nowa trasa jest dluzsza lub rowna
[4 2 1 0 3]
10

18 iteracja
nowa trasa jest dluzsza